In [6]:
import regex
import numpy as np
import pandas as pd
# Gensim
import gensim
import gensim.corpora as corpora
from gensim.utils import simple_preprocess

# spacy for lemmatization
import spacy

# Plotting tools
import pyLDAvis
from wordcloud import WordCloud

In [7]:
df = pd.read_csv("dataset/Text.csv",encoding='latin1')

C:\Users\wuchu\anaconda3\lib\site-packages\pandas\core\indexes\base.py:395: DeprecationWarning: `np.bool` is a deprecated alias for the builtin `bool`. To silence this warning, use `bool` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.bool_` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  np.ndarray,
C:\Users\wuchu\anaconda3\lib\site-packages\pandas\core\indexes\base.py:395: DeprecationWarning: `np.bool` is a deprecated alias for the builtin `bool`. To silence this warning, use `bool` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.bool_` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  np.ndarray,
C:\Users\wuchu\anaconda3\lib\site-packages\pandas\core\indexes\base.py:395: Deprec

In [12]:
df.shape

(19883, 7)

In [13]:
df.head()

AttributeError: 'NoneType' object has no attribute 'items'

   Unnamed: 0                                          post_text  followers  \
0           0  It's just over 2 years since I was diagnosed w...         84   
1           1  It's Sunday, I need a break, so I'm planning t...         84   
2           2  Awake but tired. I need to sleep but my brain ...         84   
3           3  RT @SewHQ: #Retro bears make perfect gifts and...         84   
4           4  Itâs hard to say whether packing lists are m...         84   

   friends  favourites  statuses  label  
0      211         251       837      1  
1      211         251       837      1  
2      211         251       837      1  
3      211         251       837      1  
4      211         251       837      1  

In [11]:
data = df[df['label']==1]
data

AttributeError: 'NoneType' object has no attribute 'items'

      Unnamed: 0                                          post_text  \
0              0  It's just over 2 years since I was diagnosed w...   
1              1  It's Sunday, I need a break, so I'm planning t...   
2              2  Awake but tired. I need to sleep but my brain ...   
3              3  RT @SewHQ: #Retro bears make perfect gifts and...   
4              4  Itâs hard to say whether packing lists are m...   
...          ...                                                ...   
9939        9995  RT @CSLewisU: "We all agreed that forgiveness ...   
9940        9996  Can't believe it's been a year. #blastyblast #...   
9941        9997  One year ago, a journey began. I never knew th...   
9942        9998  I found some new hammocking friends in the par...   
9943        9999              @xanabby you'll grow to like it. ð   

      followers  friends  favourites  statuses  label  
0            84      211         251       837      1  
1            84      211         25

In [ ]:
import nltk;
from nltk.corpus import stopwords
stop_words = stopwords.words('english')

In [ ]:
def sent_to_words(sentences):
    for sentence in sentences:
        yield(gensim.utils.simple_preprocess(str(sentence), deacc=True))  # deacc=True removes punctuations
        
data_words = list(sent_to_words(data['post_text']))

print(data_words[0])

In [ ]:
bigram = gensim.models.Phrases(data_words, min_count=5, threshold=100)
bigram_mod = gensim.models.phrases.Phraser(bigram)

print(bigram_mod[data_words[0]])

In [ ]:
def remove_stopwords(texts):
    return [[word for word in simple_preprocess(str(doc)) if word not in stop_words] for doc in texts]

def make_bigrams(texts):
    return [bigram_mod[doc] for doc in texts]


def lemmatization(texts, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV']):
    texts_out = []
    for sent in texts:
        doc = nlp(" ".join(sent)) 
        texts_out.append([token.lemma_ for token in doc if token.pos_ in allowed_postags])
    return texts_out

In [ ]:
data_words_nostops = remove_stopwords(data_words)

data_words_bigrams = make_bigrams(data_words_nostops)

nlp = spacy.load('en_core_web_sm', disable=['parser', 'ner'])

data_lemmatized = lemmatization(data_words_bigrams, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV'])

print(data_lemmatized[0])

In [ ]:
id2word = corpora.Dictionary(data_lemmatized)

texts = data_lemmatized

corpus = [id2word.doc2bow(text) for text in texts]

print(corpus[0])

In [ ]:
lda_model = gensim.models.ldamodel.LdaModel(corpus=corpus,
                                           id2word=id2word,
                                           num_topics=3, 
                                           random_state=100,
                                           update_every=1,
                                           chunksize=100,
                                           passes=10,
                                           alpha='auto',
                                           per_word_topics=True)
print(lda_model.print_topics())
doc_lda = lda_model[corpus]

In [ ]:
import pyLDAvis.gensim_models
pyLDAvis.enable_notebook()
vis = pyLDAvis.gensim_models.prepare(lda_model, corpus, id2word)
vis